# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [32]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------
from sklearn.feature_selection import SelectKBest, chi2 ,mutual_info_classif
from sklearn.svm import SVC 
from numpy import mean ,std
from sklearn.model_selection import  cross_validate ,RepeatedKFold , RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [33]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [4]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
# import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [34]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [47]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [36]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

### Function for cleaning the text and setup the training set
- Cleaning numbers and spaces from the text

In [37]:
# Remove numbers , spaces and leading and trailing spaces (Spaces and numbers)
def text_cleaning(text):
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    
    return text

### Models and parameters 
- Defining a Dictionary with all the models we will be training
- Defining a Dictionary with all the parameters we will use to tune the models

In [38]:
models_list = {
    'Perceptron':Perceptron(),
    'SGDClassifier':SGDClassifier(),
    'lr': LogisticRegression(),
    'svm': SVC(),
    'knn': KNeighborsClassifier(),
    'cart': DecisionTreeClassifier(),
    'LinearSVC':LinearSVC(),
    'MLPClassifier':MLPClassifier()
}

params = {
    'Perceptron': {'alpha': [0.0001, 0.05], 'penalty': [None, 'l2', 'l1', 'elasticnet']},
    'SGDClassifier': {'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
                            'penalty': ['l2', 'l1', 'elasticnet'],
                            'alpha': [0.0001, 0.05]} ,
    'lr': {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']},
    'svm': {'C': [0.01, 0.1, 1, 10, 100], 'kernel': ['linear', 'rbf']},
    'knn': {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']},
    'cart': {'max_depth': [3, 5, 7], 'criterion': ['gini', 'entropy']},
    'LinearSVC': {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2'], 'dual': [False]},
    'MLPClassifier': {'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
                        'activation': ['tanh', 'relu'],
                        'solver': ['sgd', 'adam'],
                        'alpha': [0.0001, 0.05],
                        'learning_rate': ['constant','adaptive']}
}

### Functions for models and parameters
- Defining a function that returns a dictionary with all the models with the input parameters
- Defining a function that finds the best parameters for each model using GridSearchCV

In [39]:
def get_models(params):
    models = dict()
    models['Perceptron'] = Perceptron(**params['Perceptron'])
    models['SGDClassifier'] = SGDClassifier(**params['SGDClassifier'])
    models['lr'] = LogisticRegression(**params['lr'])
    models['svm'] = SVC(**params['svm'])
    models['knn'] = KNeighborsClassifier(**params['knn'])
    models['cart'] = DecisionTreeClassifier(**params['cart'])
    models['LinearSVC'] = LinearSVC(**params['LinearSVC'])
    models['MLPClassifier'] = MLPClassifier(**params['MLPClassifier'])

    return models

# finding the best parameters for each model
def find_best_params(models , X_train_selected ,y_train ):
    best_params = {}
    for name, model in models.items():
        clf = GridSearchCV(model, params[name], cv=5, n_jobs=-1) #Using GridSearch to find best params
        clf.fit(X_train_selected, y_train)
        best_params[name] = clf.best_params_
        
    return best_params

### Model Evaluation
- Defining a function that evaluate the model and returns its score

In [40]:
# evaluate models using cross-validation with diffrent params
def model_evaluation(model,X_train_selected, X_test_selected  ,y_train, y_test):
    model_train = model.fit(X_train_selected , y_train)

    cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
    scores = cross_val_score(model_train, X_test_selected, y_test, scoring='accuracy', cv=cv, n_jobs=-1  )

    y_pred = model_train.predict(X_test_selected)
    print(model_train)
    
    f1_male = f1_score(y_test, y_pred, pos_label='m')
    f1_female = f1_score(y_test, y_pred, pos_label='f')
    average_f1=(f1_male + f1_female)/2
    
    evalutaion = dict()
    evalutaion['Accuracy'] = mean(scores)
    evalutaion['F1 Male Score'] = f1_male
    evalutaion['F1 Female Score'] = f1_female
    evalutaion['scores'] = scores
    evalutaion['average_f1'] = average_f1
    evalutaion['model'] = model_train
    
    return evalutaion

### Results
- Setup the training set by applying the text_cleaning, vectorizer, SelectKBest, MinMaxScaler
- Iterating through each model and printing score results

In [41]:
df = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df['story'] = df['story'].apply(text_cleaning) #cleaning the story column
    
X_train, X_test, y_train, y_test = train_test_split(df['story'], df['gender'], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(min_df= 5 ,ngram_range= (1,1)) #TfidVectorizer setup

X_train_after_vect = vectorizer.fit_transform(X_train)
X_test_after_vect = vectorizer.transform(X_test)

selector = SelectKBest(mutual_info_classif, k=1000) #Select features according to the k highest scores using mutual info
selector.fit(X_train_after_vect, y_train)

X_train_after_selection = selector.transform(X_train_after_vect) # Apply feature selection
X_test_after_selection = selector.transform(X_test_after_vect) # Apply feature selection
    
scaler = MinMaxScaler()
X_train_final = scaler.fit_transform(X_train_after_selection.toarray())
X_test_final = scaler.transform(X_test_after_selection.toarray())

best_params = find_best_params(models_list , X_train_final ,y_train ) #finding best params

models = get_models(best_params) #getting models with best params

# models evaluation and printing results
score = {}
best_score = 0
best_model = ''
for name, model in models.items():
    scores = model_evaluation(model, X_train_final, X_test_final, y_train, y_test)
    score[name] = scores
    if(best_score < scores['average_f1']):
        best_score = scores['average_f1']
        best_model = name
    print('%s Accuracy: %.3f' % (name, scores['Accuracy']))
    print('F1 Female Score : %.3f' % (scores['F1 Female Score']))
    print('F1 Male Score : %.3f' % (scores['F1 Male Score']))
    print('\033[1m' + 'average_F1 Score : %.3f\n\n-------------\n' % (scores['average_f1']) + '\033[0m')
    
print('The best model is: %s' % (best_model))
print('With %.3f average_f1 score.' % (best_score))


SelectKBest(k=1000,
            score_func=<function mutual_info_classif at 0x000002B30CF715E0>)

Perceptron()
Perceptron Accuracy: 0.771
F1 Female Score : 0.629
F1 Male Score : 0.888
average_F1 Score : 0.758

-------------

SGDClassifier(loss='modified_huber', penalty='elasticnet')
SGDClassifier Accuracy: 0.744
F1 Female Score : 0.517
F1 Male Score : 0.885
average_F1 Score : 0.701

-------------

LogisticRegression(C=1)
lr Accuracy: 0.759
F1 Female Score : 0.444
F1 Male Score : 0.879
average_F1 Score : 0.662

-------------

SVC(C=0.1, kernel='linear')
svm Accuracy: 0.766
F1 Female Score : 0.456
F1 Male Score : 0.873
average_F1 Score : 0.665

-------------

KNeighborsClassifier(weights='distance')
knn Accuracy: 0.753
F1 Female Score : 0.000
F1 Male Score : 0.860
average_F1 Score : 0.430

-------------

DecisionTreeClassifier(criterion='entropy', max_depth=3)
cart Accuracy: 0.697
F1 Female Score : 0.423
F1 Male Score : 0.880
average_F1 Score : 0.652

-------------

LinearSVC(C=1, dual=False)
LinearSVC Accuracy: 0.771
F1 Female Score : 0.478
F1 Male Score : 0.851
average_F1 Score : 0

### Predicting the test data
After we saw the Perceptron model gave us the best score we will choose it to be the best model to predict the test data
We will show the other models predictions as well.

In [46]:
#Setting up the test data
df = pd.read_csv(test_filename, index_col=None, encoding='utf-8')
df['story'] = df['story'].apply(text_cleaning) #cleaning the story column
vectorizer = TfidfVectorizer(min_df=5, ngram_range=(1,1)) #TFidVectorizer setup

X_testdata = df['story']
X_testdata_after_vect = vectorizer.fit_transform(X_testdata)
        
selector = SelectKBest(mutual_info_classif, k=1000) #Select features according to the k highest scores using mutual info
X_testdata_after_selection = selector.fit_transform(X_testdata_after_vect, np.zeros(X_testdata_after_vect.shape[0]))

scaler = MinMaxScaler()
X_test_final = scaler.fit_transform(X_testdata_after_selection.toarray())
story_column = df['story']

#Test data prediction
df = pd.DataFrame()
df_predict_dict = {}
for mod in score:
    y_pred = score[mod]['model'].predict(X_test_final)
    df_predict = pd.DataFrame({'story': story_column, 'predicted gender': y_pred})
    df = df.append(df_predict['predicted gender'].value_counts(),ignore_index=True)
    if(mod == 'Perceptron'):
        df_predict_dict = df_predict
        
df = df.rename({0: 'Perceptron', 1: 'SGDClassifier', 2: 'lr', 3: 'svm', 4: 'knn', 5: 'cart', 6: 'LinearSVC', 7: 'MLPClassifier'})
display(df)

display(df_predict_dict)
df_predicted = df_predict_dict


,m,f
Perceptron,187,136.000
SGDClassifier,242,81.000
lr,275,48.000
svm,291,32.000
knn,323,NaN
cart,286,37.000
LinearSVC,187,136.000
MLPClassifier,184,139.000


,story,predicted gender
0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m
1,הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת י...,m
2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m
3,רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי אני ...,f
4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,m
...,...,...
318,בשנה האחרונה הרגשתי די תקוע בעבודה השגרה הפכה ...,m
319,אני ואילן חברים טובים מזה שנה תמיד חלמנו לפתח...,f
320,מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...,f
321,לפני מספר חודשים בשיא התחלואה של הגל השני עמית...,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [23]:
df_predicted.to_csv('classification_results.csv',index=False)